# PanAnalysis Test Data

## Import required modules

Import the modules required for this solution

In [1]:
import pyodbc
import pandas as pd
import os

## Load data into dataframe

In [2]:
# Database connection data - may or may not be used...
s_server = '169.254.250.189\SQLEXPRESS' 
s_database = 'PanAnalysis'
s_user = 'sa'
s_password = 'Demo'
s_connection = 'DRIVER={SQL Server Native Client 11.0};' 
s_connection += 'SERVER=' + s_server + ';DATABASE=' + s_database +';'
s_connection += 'UID=' + s_user +';PWD=' + s_password

s_savefile = 'pan_data.pkl'

# Read in the data set to a data frame
if not os.path.exists(s_savefile):
    # Read from the database
    print('Reading from live database...')
    dbconn = pyodbc.connect(s_connection)
    print('Database connection made...')
    s_tmpsql = 'select * from zTempPanAnalysis'
    df_initial = pd.read_sql_query(s_tmpsql, dbconn)
    # Now save it out for later
    df_initial.to_pickle(s_savefile)
else:
    # We have read the data before, so can pull this in from a pickle file
    print('Reading from cached data...')
    df_initial = pd.read_pickle(s_savefile)

Reading from cached data...


In [3]:
# Check we have this OK
df_initial[0:3]

,ID,Incident,IncidentType,CustomerId,LedgerCode,BusinessType,SiteId,MachinesOnSite,SoleMachine,Manufacturer,...,AttendOnSite,AttendTimeValid,CallMinutes,MinutesToAttend,MinutesToAttendValid,PostCodeArea,FirstEngineer,SymptomCodeId,SymptomDescription,Repeat
0,1,145001,BREAKDOWN,12473,S798,Education - Junior/Primary school,21019,4,False,TOSHIBA,...,True,True,52,769,True,DE23,BEN,S44,Connectivity,False
1,2,145002,BREAKDOWN,11546,F119,Education - Junior/Primary school,20497,1,True,TOSHIBA,...,True,True,161,814,True,BD10,SERING,S44,Connectivity,False
2,4,145004,BREAKDOWN,10751,BS036,Education - Junior/Primary school,23322,2,False,TOSHIBA,...,True,True,70,688,True,BD13,RONNIE,S44,Connectivity,False


## Shape the data ready for model building
We will need:
- Categorical values to be one-hot encoded - this is for decision tree work and deep learning
- Scaling and normalisation of scalar quantities - this is for deep learning only

Objective is to produce a data frame for decision trees and a data frame for deep learning.

The data frame for deep learning may require data augmentation to balance the training set (REPEATS / non-REPEATS)

### Decision tree data frame
Prepare a data frame for the decision tree work

In [ ]:
# Make a copy of the main data frame
df_tree = df_initial.copy()

Categoricals to encode are:
- Business type
- Manufacturer
- Product Id (this is going to generate a *lot* of columns)
- Device Type
- LastOtherCallType
- CreatedBy (lots)
- PostCodeArea (another large set)
- FirstEngineer
- SymptomCodeId (lots)

In [ ]:
# One-hot encoding for engineers
one_hot = pd.get_dummies(df_tree['BusinessType'])
print(one_hot[0:10])

In [ ]:
# Now do the join to augment the data frame
df_tree = df_tree.join(one_hot)
df_tree[0:3]

Now do the same process for each of the fields we want to one-hot encode

Now create the target value for the model

In [ ]:
y_repeat = df_tree['IncidentType'].map(lambda x: x == 'REPEAT')

In [ ]:
# Just show a few values - there will be few 'True' values as REPEATS are sparse
print(df_tree['IncidentType'][0:5])
print(y_repeat[0:5])

Now drop off fields which are not going to be used in the model process.

Drop the field which have been one-hot encoded.

Fields which are not useful and can also be dropped are:
- ID
- Incident (incident code)
- 

In [ ]:
print(df_tree.columns)

## (Below here is nurdling...)

In [ ]:
# Simple database query test
s_tmpsql = 'select top 10 * from zTempPanAnalysis'
dbcursor = dbconn.cursor()
dbcursor.execute(s_tmpsql)
dbrecordset = dbcursor.fetchall()
for record in dbrecordset:
    print('Incident ID :', record.Incident)

print('\nShould have printed 10 6-digit incident IDs.')

## Load some data into a dataframe

In [ ]:
data_array = []
for record in dbrecordset:
    data_array += [[record.Incident, record.IncidentType, record.CustomerId]]
print('Data array looks like:\n')
print(data_array)

In [ ]:
df = pd.DataFrame(data_array, columns=['Incident','IncidentType','CustomerId'])
print('Data frame looks like:\n')
print(df)

In [ ]:
# Get the columns from the recordset
s_cols = [col[0] for col in dbcursor.description]
print('Columns list:\n', s_cols)

In [ ]:
# This should fetch some rows and give the sensible column names
s_tmpsql2 = 'select top 10 * from zTempPanAnalysis'
df2 = pd.read_sql_query(s_tmpsql2, dbconn)

In [ ]:
print(df2)


In [ ]:
# Can now mess about with the dataframe
y = df['IncidentType']
print('Extracted:\n',y)

In [ ]:
data = df2
del data['IncidentType']
print('Feature data:\n',data)

In [ ]:
y_repeat = y.map(lambda x: x == 'REPEAT')
print(y_repeat)
print(y)

In [ ]:
df_times = df2[['AttendHour','CallMinutes','MinutesToAttend']].copy()
print (df_times)

In [ ]:
# Now build a decision tree classifier on this data
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
dtree.fit(df_times,y_repeat)

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz

In [ ]:
# Note: had to install this new package
import pydotplus

In [ ]:
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
## Run on the whole dataset
# load in the data
s_tmpsql_all = 'select * from zTempPanAnalysis'
data_all = pd.read_sql_query(s_tmpsql_all, dbconn)
# get the repeats
y_all = data_all['IncidentType']
# get rid of the data not required
del data_all['IncidentType']
# encode the repeat data for predicion
y_repeat_all = y_all.map(lambda x: x == 'REPEAT')
# copy out the data we want
df_times_all = data_all[['AttendHour','CallMinutes','MinutesToAttend']].copy()
# replace any remaining NaN values with -1, to avoid breaking the algorithm
df_times_all = df_times_all.fillna(-1)
#print (df_times_all)
# fit a classifier
dtree_all=DecisionTreeClassifier(min_samples_split=500, max_depth=5)
dtree_all.fit(df_times_all,y_repeat_all)
# visualise
dot_data = StringIO()
export_graphviz(dtree_all, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
#df_times_all[0:10]
print(dtree_all.decision_path(df_times_all[0:10]))

## Need to do some management of the data - e.g. categoricals to IDs etc

1. Convert categorical data to IDs - e.g. engineer names, postcodes - consider if one-hot encoding is better here
2. Remove fields we don't need - e.g. redundant datatime values, incident IDs, serial numbers


In [ ]:
# Count the distinct values in a categorical column
df_categorical = data_all.copy()

In [ ]:
df_categorical['BusinessType'] = df_categorical['BusinessType'].astype('category')
df_categorical.dtypes

In [ ]:
df_categorical['BusinessType']

In [ ]:
# Creates a new column with a categorical ID generated from the existing categorical column
df_categorical['BusinessType_Categorical'] = df_categorical['BusinessType'].cat.codes

In [ ]:
df_categorical['BusinessType']
df_categorical['BusinessType_Categorical']

In [ ]:
data_all

In [ ]:
df_categorical.dtypes

In [ ]:
# One-hot encoding for engineers
one_hot = pd.get_dummies(data_all['FirstEngineer'])

In [ ]:
one_hot[0:10]

In [ ]:
# Now join the one-hot encoded columns to the main dataframe
df_coded = df_categorical.join(one_hot)

In [ ]:
df_coded[0:10]

In [ ]:
df_categorical[0:10]

In [ ]:
data_all[0:10]